In [47]:
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [48]:
# Initialize geolocator
geolocator = Nominatim(user_agent="geoapiExercises")

In [50]:
# Define sample latitude and longitude coordinates
Latitude = "40.74677541"
Longitude = "-73.78086727"

# Reverse geocode the coordinates to get the location information
location = geolocator.reverse(Latitude+","+Longitude)
 
# Display the location information
print(location)

53-50, Hollis Court Boulevard, Queens, City of New York, New York, 11365, United States


In [52]:
#pip install geopandas

In [78]:
# Read the shapefile containing NYC neighborhoods
nyc_neighborhoods = gpd.read_file("nynta2020.shp")

In [145]:
# Read the sample crime data CSV file
data = pd.read_csv("sample_crime_data_2_2021.csv")

In [174]:
# Create a Point geometry from X and Y coordinates in the crime data
points = data.apply(lambda row:Point(row['X_COORD_CD'],row['Y_COORD_CD']), axis=1)

In [175]:
points

0         POINT (1019853 241853)
1         POINT (1021342 237277)
2         POINT (1032310 252550)
3         POINT (1044969 211420)
4         POINT (1008383 225051)
                   ...          
439620    POINT (1004317 255023)
439621     POINT (987309 190698)
439622    POINT (1049939 188315)
439623     POINT (985720 214562)
439624    POINT (1004950 179926)
Length: 439625, dtype: object

In [176]:
# Create a GeoDataFrame from the Points and assign index from crime data
pointsDF = gpd.GeoDataFrame(geometry=list(points1),
                                  index=list(data['CMPLNT_NUM']))
# Create a GeoDataFrame from the neighborhood polygons
polysgdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(nyc_neighborhoods['geometry']))

In [177]:
# Display the Points GeoDataFrame
pointsDF

,geometry
708871766,POINT (1019853.000 241853.000)
123171799,POINT (1021342.000 237277.000)
725872999,POINT (1032310.000 252550.000)
432371969,POINT (1044969.000 211420.000)
217646811,POINT (1008383.000 225051.000)
...,...
631903360,POINT (1004317.000 255023.000)
695555770,POINT (987309.000 190698.000)
147985546,POINT (1049939.000 188315.000)
153855183,POINT (985720.000 214562.000)


In [188]:
# Perform a spatial join to associate each point with a neighborhood
joinDF=gpd.sjoin(pointsDF, polysgdf, how='left',op="within")

C:\Users\SaraM\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\SaraM\AppData\Local\Temp\ipykernel_23756\1510000659.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:2263

  joinDF=gpd.sjoin(pointsDF, polysgdf, how='left',op="within")


In [189]:
joinDF

,geometry,index_right
708871766,POINT (1019853.000 241853.000),95.0
123171799,POINT (1021342.000 237277.000),96.0
725872999,POINT (1032310.000 252550.000),103.0
432371969,POINT (1044969.000 211420.000),210.0
217646811,POINT (1008383.000 225051.000),156.0
...,...,...
631903360,POINT (1004317.000 255023.000),152.0
695555770,POINT (987309.000 190698.000),5.0
147985546,POINT (1049939.000 188315.000),219.0
153855183,POINT (985720.000 214562.000),127.0


In [190]:
# Drop rows with missing neighborhood information
joinDF.dropna(inplace=True, axis=0)

In [191]:
joinDF.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 439584 entries, 708871766 to 744048367
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   geometry     439584 non-null  geometry
 1   index_right  439584 non-null  float64 
dtypes: float64(1), geometry(1)
memory usage: 10.1 MB


In [212]:
# Extract neighborhood names for each point
neighborhoods= []
for i in list(joinDF.index):
    neighborhoods.append(nyc_neighborhoods['NTAName'][int(joinDF['index_right'][i])])

In [213]:
neighborhoods

['Soundview-Bruckner-Bronx River',
 'Soundview-Clason Point',
 'Co-op City',
 'Auburndale',
 'Astoria (North)-Ditmars-Steinway',
 'Crown Heights (North)',
 'Greenwich Village',
 'Van Cortlandt Park',
 'Morrisania',
 'Soundview-Bruckner-Bronx River',
 'Midtown South-Flatiron-Union Square',
 'Midtown-Times Square',
 'East Elmhurst',
 'Upper West Side-Lincoln Square',
 'Castle Hill-Unionport',
 'Park Slope',
 'East Harlem (North)',
 'Financial District-Battery Park City',
 'Flatlands',
 'Concourse-Concourse Village',
 'Soundview-Clason Point',
 'Marine Park-Mill Basin-Bergen Beach',
 'Middle Village',
 'Bedford-Stuyvesant (East)',
 'Mott Haven-Port Morris',
 'Highbridge',
 'Jamaica Hills-Briarwood',
 'Flatlands',
 'SoHo-Little Italy-Hudson Square',
 'Midtown-Times Square',
 'Soundview-Bruckner-Bronx River',
 'East Flatbush-Remsen Village',
 'Soundview-Bruckner-Bronx River',
 'Pelham Gardens',
 'Crown Heights (North)',
 'Riverdale-Spuyten Duyvil',
 'Soundview-Clason Point',
 'Morningside H

In [215]:
# Add the neighborhoods column to the joined GeoDataFrame
joinDF['neighborhoods']=neighborhoods

In [216]:
joinDF

,geometry,index_right,neighborhoods
708871766,POINT (1019853.000 241853.000),95.0,Soundview-Bruckner-Bronx River
123171799,POINT (1021342.000 237277.000),96.0,Soundview-Clason Point
725872999,POINT (1032310.000 252550.000),103.0,Co-op City
432371969,POINT (1044969.000 211420.000),210.0,Auburndale
217646811,POINT (1008383.000 225051.000),156.0,Astoria (North)-Ditmars-Steinway
...,...,...,...
631903360,POINT (1004317.000 255023.000),152.0,Inwood
695555770,POINT (987309.000 190698.000),5.0,Downtown Brooklyn-DUMBO-Boerum Hill
147985546,POINT (1049939.000 188315.000),219.0,St. Albans
153855183,POINT (985720.000 214562.000),127.0,Chelsea-Hudson Yards


In [217]:
# Assign the original index as a column in the joined GeoDataFrame
joinDF['CMPLNT_NUM']=joinDF.index

In [220]:
# Reset the index of the joined GeoDataFrame
joinDF.reset_index(drop=True)

,geometry,index_right,neighborhoods,CMPLNT_NUM
0,POINT (1019853.000 241853.000),95.0,Soundview-Bruckner-Bronx River,708871766
1,POINT (1021342.000 237277.000),96.0,Soundview-Clason Point,123171799
2,POINT (1032310.000 252550.000),103.0,Co-op City,725872999
3,POINT (1044969.000 211420.000),210.0,Auburndale,432371969
4,POINT (1008383.000 225051.000),156.0,Astoria (North)-Ditmars-Steinway,217646811
...,...,...,...,...
439579,POINT (1004317.000 255023.000),152.0,Inwood,631903360
439580,POINT (987309.000 190698.000),5.0,Downtown Brooklyn-DUMBO-Boerum Hill,695555770
439581,POINT (1049939.000 188315.000),219.0,St. Albans,147985546
439582,POINT (985720.000 214562.000),127.0,Chelsea-Hudson Yards,153855183


In [221]:
# Save the joined GeoDataFrame to a CSV file
joinDF.to_csv('sample_crime_data_neighborhoods.csv', index=False)

In [222]:
# Group the data by neighborhoods and count the number of crime incidents
joinDF.groupby(['neighborhoods'])['CMPLNT_NUM'].count()

neighborhoods
Allerton                                  1705
Alley Pond Park                             46
Annadale-Huguenot-Prince's Bay-Woodrow     834
Arden Heights-Rossville                    496
Astoria (Central)                         2032
                                          ... 
Windsor Terrace-South Slope                503
Woodhaven                                 1248
Woodlawn Cemetery                          177
Woodside                                  1819
Yankee Stadium-Macombs Dam Park            357
Name: CMPLNT_NUM, Length: 256, dtype: int64